<a href="https://colab.research.google.com/github/romiebanerjee/mmsegmentation-bayes/blob/main/mmlab_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMSegmentation-bayes tools tutorial on Colab

In this tutorial, we will introduce the following content:

* How to install mmseg-bayes
* Fetch a pre-trained mmseg model
* Prepare data for Fisher estimation
* Make the BNN model from pre-trained model and Fisher
* Run example to compute post-hoc uncertainties

## Install mmseg-bayes

Before using mmseg-bayes, we need to prepare the environment with the following steps:

1. Install Python, CUDA, C/C++ compiler and git
2. Install PyTorch (CUDA version)
3. Install mmcv
4. Install mmengine-bayes

Because this tutorial is on Google Colab, and the basic environment has been completed, we can skip the first two steps.

### Check Environment

In [14]:
%cd /content

/content


In [15]:
!pwd

/content


In [16]:
#check nvcc version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [17]:
# Check GCC version
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [18]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


In [ ]:
import torch
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

### Install MMCV

MMCV is the basic package of all OpenMMLab packages. We have pre-built wheels on Linux, so we can download and install them directly.

Please pay attention to PyTorch and CUDA versions to match the wheel.

In the above steps, we have checked the version of PyTorch and CUDA, and they are 2.6.0 and 12.4 respectively, so we need to choose the corresponding wheel.

In addition, we can also install the full version of mmcv (mmcv-full). It includes full features and various CUDA ops out of the box, but needs a longer time to build.

In [9]:
!pip install ftfy cityscapesscripts

In [ ]:
# For CUDA 11.8 + PyTorch 2.0+
!pip install -U openmim
!mim install mmcv #this might take upto 40 mins!

Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
Could not fetch URL https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='download.openmmlab.com', port=443): Max retries exceeded with url: /mmcv/dist/cu124/torch2.6.0/index.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1016)'))) - skipping
  Using cached mmcv-2.2.0.tar.gz (479 kB)
  Preparing metadata (setup.py) ... done


### Clone and install mmengine-bayes and mmsegmentation-bayes

In [ ]:
!git clone https://github.com/romiebanerjee/mmengine-bayes
!pip install -e mmengine-bayes/.

In [ ]:
!git clone https://github.com/romiebanerjee/mmsegmentation-bayes
!pip install -e mmsegmentation-bayes/.

In [ ]:
import sys
sys.path.append('/content/mmsegmentation-bayes/')
sys.path.append('/content/mmengine-bayes/')
import mmcv, mmengine, mmseg

## Laplace Approximation with KFAC

### Download a pre-trained model from MMSeg

In [ ]:
!wget --no-check-certificate https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth


In [ ]:
import torch
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/mmsegmentation-bayes/checkpoints/
!cp -rv /content/drive/MyDrive/data/checkpoints/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth /content/mmsegmentation-bayes/checkpoints/
!cp -rv /content/drive/MyDrive/data/work_dirs/deeplabv3plus_r50-d8_4xb2-40k_cityscapes-512x1024/kfac_state_iter_latest.pt /content/mmsegmentation-bayes/checkpoints/

### Prepare training data for estimating Fisher

In [ ]:
# !pwd
# !mkdir /content/mmsegmentation-bayes/data/
# !ln -s /content/drive/MyDrive/data/cityscapes /content/mmsegmentation-bayes/data/

In [ ]:
# !pwd
# --nproc means 8 process for conversion, which could be omitted as well.
# !python mmsegmentation-bayes/tools/dataset_converters/cityscapes.py mmsegmentation-bayes/data/cityscapes --nproc 8

### Estimate Fisher

In [ ]:
%cd mmsegmentation-bayes
!pwd
!python tools/fisher.py './configs/deeplabv3plus/deeplabv3plus_r50-d8_4xb2-40k_cityscapes-512x1024.py' --cfg-options load_from="checkpoints/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth" device="cuda"

## Use BNN for predictive *distributions*

In [ ]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
from mmengine.curvature import KFAC
from mmseg.apis import SegGLM

In [ ]:
config_path = '/content/mmsegmentation-bayes/configs/deeplabv3plus/deeplabv3plus_r50-d8_4xb2-40k_cityscapes-512x1024.py'
model_path =  '/content/mmsegmentation-bayes/checkpoints/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth'
curvature_checkpoint = '/content/mmsegmentation-bayes/checkpoints/kfac_state_iter_latest.pt'

In [ ]:

myglm = SegGLM(model_checkpoint = model_path, config = config_path, device = device, curvature_checkpoint= curvature_checkpoint)

In [ ]:
# show the results
from matplotlib import pyplot as plt

img = '/content/mmsegmentation-bayes/demo/demo.png'
output_path = '/content/mmsegmentation-bayes/demo/seg_output.png'

result = inference_model(myglm.model, img)
vis_result = show_result_pyplot(myglm.model, img, result, show=False)
plt.imshow(vis_result)

In [ ]:
seg_unc = myglm.mcglm_predictive(image = img, eps = 1e-6, iters = 10, eig_idx = None)